In [1]:
import pymorphy2
import os
from pymystem3 import Mystem  
m = Mystem() 
import re
from math import log
import operator

Обратные индексы

In [2]:
stopw = ['a', 'about', 'all', 'am', 'an', 'and', 'any', 'are', 'as', 'at', 'be', 'been', 'but', 'by', 'can', 'could', 'do', 'for', 'from', 'has', 'have', 'i', 'if', 'in', 'is', 'it', 'me', 'my', 'no', 'not', 'of', 'on', 'one', 'or', 'so', 'that', 'the', 'them', 'there', 'they', 'this', 'to', 'was', 'we', 'what', 'which', 'will', 'with', 'would', 'you', 'а', 'будем', 'будет', 'будете', 'будешь', 'буду', 'будут', 'будучи', 'будь', 'будьте', 'бы', 'был', 'была', 'были', 'было', 'быть', 'в', 'вам', 'вами', 'вас', 'весь', 'во', 'вот', 'все', 'всё', 'всего', 'всей', 'всем', 'всём', 'всеми', 'всему', 'всех', 'всею', 'всея', 'всю', 'вся', 'вы', 'да', 'для', 'до', 'его', 'едим', 'едят', 'ее', 'её', 'ей', 'ел', 'ела', 'ем', 'ему', 'емъ', 'если', 'ест', 'есть', 'ешь', 'еще', 'ещё', 'ею', 'же', 'за', 'и', 'из', 'или', 'им', 'ими', 'имъ', 'их', 'к', 'как', 'кем', 'ко', 'когда', 'кого', 'ком', 'кому', 'комья', 'которая', 'которого', 'которое', 'которой', 'котором', 'которому', 'которою', 'которую', 'которые', 'который', 'которым', 'которыми', 'которых', 'кто', 'меня', 'мне', 'мной', 'мною', 'мог', 'моги', 'могите', 'могла', 'могли', 'могло', 'могу', 'могут', 'мое', 'моё', 'моего', 'моей', 'моем', 'моём', 'моему', 'моею', 'можем', 'может', 'можете', 'можешь', 'мои', 'мой', 'моим', 'моими', 'моих', 'мочь', 'мою', 'моя', 'мы', 'на', 'нам', 'нами', 'нас', 'наса', 'наш', 'наша', 'наше', 'нашего', 'нашей', 'нашем', 'нашему', 'нашею', 'наши', 'нашим', 'нашими', 'наших', 'нашу', 'не', 'него', 'нее', 'неё', 'ней', 'нем', 'нём', 'нему', 'нет', 'нею', 'ним', 'ними', 'них', 'но', 'о', 'об', 'один', 'одна', 'одни', 'одним', 'одними', 'одних', 'одно', 'одного', 'одной', 'одном', 'одному', 'одною', 'одну', 'он', 'она', 'оне', 'они', 'оно', 'от', 'по', 'при', 'с', 'сам', 'сама', 'сами', 'самим', 'самими', 'самих', 'само', 'самого', 'самом', 'самому', 'саму', 'свое', 'своё', 'своего', 'своей', 'своем', 'своём', 'своему', 'своею', 'свои', 'свой', 'своим', 'своими', 'своих', 'свою', 'своя', 'себе', 'себя', 'собой', 'собою', 'та', 'так', 'такая', 'такие', 'таким', 'такими', 'таких', 'такого', 'такое', 'такой', 'таком', 'такому', 'такою', 'такую', 'те', 'тебе', 'тебя', 'тем', 'теми', 'тех', 'то', 'тобой', 'тобою', 'того', 'той', 'только', 'том', 'томах', 'тому', 'тот', 'тою', 'ту', 'ты', 'у', 'уже', 'чего', 'чем', 'чём', 'чему', 'что', 'чтобы', 'эта', 'эти', 'этим', 'этими', 'этих', 'это', 'этого', 'этой', 'этом', 'этому', 'этот', 'этою', 'эту', 'я', 'мені', 'наші', 'нашої', 'нашій', 'нашою', 'нашім', 'ті', 'тієї', 'тією', 'тії', 'теє']

In [3]:
def indices():
    d = {}
    for root, dirs, files in os.walk('.'):
        for file in files:
            if file.endswith('.txt'):
                f = open(file, 'r', encoding='utf-8')
                lemmas = m.lemmatize(f.read())
               # for i in set(lemmas):
                for i in lemmas:
                    if i not in stopw:
                        if i not in d:
                            d[i] = [file]
                        else:
                            d[i].append(file)
                f.close()
    return(d)

In [4]:
d = indices()

Ранжирование

In [5]:
l = []
for root, dirs, files in os.walk('.'):
    for file in files:
        if file.endswith('.txt'):
            f = open(file, 'r', encoding='utf-8')
            l.append(len(f.read().split()))
            f.close()
avdl = sum(l) / float(len(l))

In [6]:
N = 1710
avdl

470.21169590643274

In [7]:
def compute_K(dl, avdl, k1, b): 
    return k1 * ((1-b) + b * (float(dl)/float(avdl)))

def score_BM25(n, fq, dl, avdl, N=1710): 
    """ 
    Okapi BM25 score 
    :param n: quantity of documents with word q 
    :param fq: frequency of word q in document D 
    :param N: size of corpus in documents 
    :param dl: length of document D 
    :param avdl: average length of document in corpus 
    :return: relevancy score 
    """ 
    k1 = 2.0 
    b = 0.75 
    K = compute_K(dl, avdl, k1, b) 
    IDF = log((N - n + 0.5) / (n + 0.5)) 
    frac = ((k1 + 1) * fq) / (K + fq) 
    return IDF * frac 

In [8]:
#okapi skore для одного слова
def query(word):
    if word not in d:
        return #??
    else:
        scores = {}
        n = len(set(d[word]))
        for i in set(d[word]):
            doc = open(i, 'r', encoding='utf-8')
            dl = len(doc.read().split())
            fq = d[word].count(i)/dl
            scores[i] = score_BM25(n, fq, dl, avdl)
            doc.close()
    return scores

In [23]:
#okapi для запроса любой длины
def mult_query(inpt):
    outp = {}
    inpt = re.sub('[,.:;!?]', '', inpt)
    for i in inpt.split():
        qi = query(i)
        for j in qi:
            if j in outp:
                outp[j] += qi[j]
            else:
                outp[j] = qi[j]
    return sorted(outp.items(), key=operator.itemgetter(1))

In [27]:
#чтобы норм выдача была со ссылками
def search(query):
    outp = []
    mq = mult_query(query)[-10:]
    for i in mq:
        url = re.findall('@url([^\n]*)\n', open(i[0], 'r', encoding='utf-8').read())
        o = (url[0], i[1])
        outp.append(o)
    return outp[::-1]

In [28]:
search('кот, собака?')

[(' http://sprostor.ru/?module=articles&action=view&id=1690',
  0.21866284768765598),
 (' http://sprostor.ru/?module=articles&action=view&id=3419',
  0.12376828916239176),
 (' http://sprostor.ru/?module=articles&action=view&id=3546',
  0.12234109023965273),
 (' http://sprostor.ru/?module=articles&action=view&id=2121',
  0.09610643293720406),
 (' http://sprostor.ru/?module=articles&action=view&id=2786',
  0.052483123859339786),
 (' http://sprostor.ru/?module=articles&action=view&id=2769',
  0.048956349262272304),
 (' http://sprostor.ru/?module=articles&action=view&id=2083',
  0.03494561333455138),
 (' http://sprostor.ru/?module=articles&action=view&id=3669',
  0.029509566105383758),
 (' http://sprostor.ru/?module=articles&action=view&id=3000',
  0.02729640653119566),
 (' http://sprostor.ru/?module=articles&action=view&id=2216',
  0.025783305073240708)]